# Model Specification for 1st-Level fMRI Analysis

Nipype provides also an interfaces to create a first level Model for an fMRI analysis. Such a model is needed to specify the study specific information, such as **condition**, their **onsets** and **durations**. For more information, make sure to check out [Model Specificaton](http://nipype.readthedocs.io/en/latest/users/model_specification.html) and [nipype.algorithms.modelgen](http://nipype.readthedocs.io/en/latest/interfaces/generated/nipype.algorithms.modelgen.html)

## Simple Example

Let's consider a simple experiment, where we have three different stimuli such as ``'faces'``, ``'houses'`` and ``'scrambled pix'``. Now each of those three conditions has different stimuli onsets, but all of them have a stimuli presentation duration of 3 seconds.

So to summarize:

    conditions = ['faces', 'houses', 'scrambled pix']
    onsets = [[0, 30, 60, 90],
              [10, 40, 70, 100],
              [20, 50, 80, 110]]
    durations = [[3], [3], [3]]
    
The way we would create this model with Nipype is almsot as simple as that. The only step that is missing is to put this all into a ``Bunch`` object. This can be done as follows:

In [1]:
from nipype.interfaces.base import Bunch

conditions = ['faces', 'houses', 'scrambled pix']
onsets = [[0, 30, 60, 90],
          [10, 40, 70, 100],
          [20, 50, 80, 110]]
durations = [[3], [3], [3]]

subject_info = Bunch(conditions=conditions,
                     onsets=onsets,
                     durations=durations)

It's also possible to specify additional regressors. For this you need to additionally specify:

- **``regressors``**: list of regressors that you want to include in the model (must correspond to the number of volumes in the functional run)
- **``regressor_names``**: name of the regressors that you want to include

## Example based on dataset

Now  let's look at a TSV file from our tutorial dataset.

In [5]:
!cat /data/ds000114/sub-01/func/sub-01_task-linebisection_events.tsv

onset	duration	weight	trial_type
24.3065	1	1.0	Incorrect_Task
25.9465	1	1.0	Correct_Task
27.5865	1	1.0	Correct_Task
29.2265	1	1.0	No_Response_Task
30.8664	1	1.0	Incorrect_Task
32.5064	1	1.0	No_Response_Task
34.1464	1	1.0	Correct_Task
35.7864	1	1.0	Incorrect_Task
37.4264	1	1.0	Correct_Task
39.0664	1	1.0	Incorrect_Task
56.973	1	1.0	Response_Control
58.613	1	1.0	No_Response_Control
60.253	1	1.0	Response_Control
61.893	1	1.0	No_Response_Control
63.533	1	1.0	Response_Control
65.173	1	1.0	Response_Control
66.8129	1	1.0	Response_Control
68.4529	1	1.0	Response_Control
70.0929	1	1.0	Response_Control
71.7329	1	1.0	No_Response_Control
89.6395	1	1.0	Correct_Task
91.2795	1	1.0	Correct_Task
92.9195	1	1.0	Incorrect_Task
94.5595	1	1.0	Incorrect_Task
96.1995	1	1.0	Correct_Task
97.8395	1	1.0	No_Response_Task
99.4795	1	1.0	Incorrect_Task
101.1194	1	1.0	Correct_Task
102.7594	1	1.0	Correct_Task
104.3994	1	1.0	Correct_Task
122.306	1	1.0	Response_Control
123.946	1	1.0	No_Response_Control
125.586	1	1.0	Respon

If the only things that we need to specify our model are the onset and the trial type, i.e. **column 0** and **column 3**, we can get them with the command:

In [8]:
import numpy as np
filename = '/data/ds000114/sub-01/func/sub-01_task-linebisection_events.tsv'
trailinfo = np.genfromtxt(filename, delimiter='\t', dtype=None, skip_header=1)
trailinfo = [[t[0], t[3]] for t in trailinfo]
trailinfo

[[24.3065, b'Incorrect_Task'],
 [25.9465, b'Correct_Task'],
 [27.586500000000001, b'Correct_Task'],
 [29.226500000000001, b'No_Response_Task'],
 [30.866399999999999, b'Incorrect_Task'],
 [32.506399999999999, b'No_Response_Task'],
 [34.1464, b'Correct_Task'],
 [35.7864, b'Incorrect_Task'],
 [37.426400000000001, b'Correct_Task'],
 [39.066400000000002, b'Incorrect_Task'],
 [56.972999999999999, b'Response_Control'],
 [58.613, b'No_Response_Control'],
 [60.253, b'Response_Control'],
 [61.893000000000001, b'No_Response_Control'],
 [63.533000000000001, b'Response_Control'],
 [65.173000000000002, b'Response_Control'],
 [66.812899999999999, b'Response_Control'],
 [68.4529, b'Response_Control'],
 [70.0929, b'Response_Control'],
 [71.732900000000001, b'No_Response_Control'],
 [89.639499999999998, b'Correct_Task'],
 [91.279499999999999, b'Correct_Task'],
 [92.919499999999999, b'Incorrect_Task'],
 [94.5595, b'Incorrect_Task'],
 [96.1995, b'Correct_Task'],
 [97.839500000000001, b'No_Response_Task'],

Before we can use the onsets, we first need to split them into the two conditions:

In [ ]:
# TODO: don't know what makes sense here...
onset1 = []
onset2 = []

for t in trailinfo:
    if 'incongruent' in t[1]:
        onset2.append(t[0])
    else:
        onset1.append(t[0])

print onset1
print onset2

The last thing we now need to to is to put this into a ``Bunch`` object and we're done:

In [ ]:
from nipype.interfaces.base import Bunch

conditions = ['congruent', 'incongruent']
onsets = [onset1, onset2]
durations = [[2], [2]]

subject_info = Bunch(conditions=conditions,
                     onsets=onsets,
                     durations=durations)